In [1]:
import pandas as pd
import numpy as np
import re
import os
from sklearn.cluster import KMeans
import ast
import itertools

In [3]:
#reading data from csv files
user_feat = pd.read_csv('user_features.csv')


num = np.arange(500,5000)
num = num[0:len(user_feat)]
user_feat['user_guid'] = num
user_feat['Calorie'] = np.random.randint(40,500, size=len(user_feat))
#print user_feat

spice_feat = pd.read_csv('spice_features.csv')

#print spice_feat
#food user rating table
food_user = pd.DataFrame(np.random.randint(low=0, high=2, size=(len(spice_feat),len(user_feat))),columns=user_feat.user_guid)
food_user.index  =  spice_feat['food_id']
#print food_user.head()



In [4]:
#Data Normalisation

def nor_age(age):
    
    if(age<=10):
        return 1
    elif(age>10 and age <=18):
        return 2
    elif(age>18 and age <=25):
        return 3
    elif(age>25 and age <=35):
        return 4
    elif(age>35 and age <=40):
        return 5
    elif(age>40 and age <=50):
        return 6
    elif(age>50 and age<=60):
        return 7
    else:
        return 8
    
    

s_dict = {"M":1,"F":5 }
l_dict = {'J&K':2, 'Haryana':5, 'Punjab':6, 'U.P.':8, 'Maharashtra':10, 'South India':15}
user_feat['Age'] = user_feat['Age'].apply(nor_age)
user_feat  = user_feat.replace({"Sex":s_dict})
user_feat  = user_feat.replace({"Location":l_dict})

In [5]:
#dataset for content based recommender
spice = np.array(spice_feat.iloc[:,0:4])
#print spice
user_rating = pd.DataFrame(np.random.randint(low=-1, high=2, size=(len(spice_feat),len(user_feat))),columns=user_feat.user_guid)
user_rating.index = spice_feat['food_id']
#user tells rating and spice tells if ingrediant is there or not.

In [6]:
#prepare dataset for 5 day window recommendation
import random
from random import randint


recommend_data = []
bought_data = []

for i in range(100):
    length = 3
    my_randoms = []
    for i in range(5):
        my_randoms.append(random.sample(xrange(6000,7000), 3))
    recommend_data.append(my_randoms)
    
    

for i in range(100):
    length = randint(0,5)
    my_randoms = []
    for i in range(5):
        my_randoms.append(random.sample(xrange(6000,7000), length))
    bought_data.append(my_randoms)
    
recommends = []
for i in range(100):
    recommends.append(random.sample(xrange(6000,7000), 3))

bought = []
for i in range(100):
    length = randint(0,5)
    bought.append(random.sample(xrange(6000,7000), length))
    

new_user_feat = pd.DataFrame(user_feat.user_guid)
new_user_feat['last_5_days_recommend'] = recommend_data
new_user_feat['last_5_days_bought'] = bought_data
new_user_feat['recommends'] = recommends
new_user_feat['bought']  = bought
new_user_feat.to_csv('new_user_feat.csv', encoding='utf-8', index=False)

In [7]:
def clustering_based(user_feat):    
    kmeans = KMeans(n_clusters = 10,random_state=0).fit(user_feat)
    user_feat['label'] = kmeans.labels_
    #print user_label
    user_feat = user_feat.sort_values(['label','Location'],ascending=[1,0])
    user = user_feat.drop_duplicates(subset=['Location','label'])
    user = user.groupby('label')['Location'].nlargest(3)
    return user,user_feat


In [8]:
def filter_food_loc(user_group,user_id,non_recom_list,user_feat,spice_feat):
    index = np.where(user_feat['user_guid']==user_id)[0][0]
    print index
    #print user_feat['label']
    user_label = user_feat.loc[index,'label']
    loc = user_group[user_label].tolist()
    newspice_feat = spice_feat.set_index('food_id')
    spice = newspice_feat[newspice_feat['Location'].isin(loc)]
    #print spice
    return filter_food_cal(spice,non_recom_list,user_feat,user_id),loc 
    
def filter_food_cal(sspice,non_recom_list,user_feat,user_id):
    index = np.where(user_feat['user_guid']==user_id)[0][0]
    sspice = sspice[sspice['NonVeg']==user_feat.loc[index,'NonVeg']]
    #print spice
    s_spice =  sspice.sort_values('count',ascending=[0])
    s_index = s_spice.index[~s_spice.index.isin(non_recom_list)]
    #print ' rec ',non_recom_list
    new_spices = s_spice[s_spice.index.isin(s_index)]
    #print new_spices
    new_spices = new_spices[new_spices['calorie']<user_feat.loc[index,'Calorie']]
    return new_spices

In [9]:
def recommendation(user_group,user_id,non_recom_list,user_feat,spice_feat):
    new_spices,loc = filter_food_loc(user_group,user_id,non_recom_list,user_feat,spice_feat)
    recommend = []
    for i in range(min(3,len(new_spices))):
        recommend.append(new_spices.index[i])

    if(len(recommend)==0):
        other_state_spice = spice_feat[spice_feat['Location'].isin(loc)==False]

        other_state_spice = other_state_spice.sort('count',ascending= [0])
        new_other_state = other_state_spice[~other_state_spice.index.isin(non_recom_list)]

        for i in range(min(3,len(new_other_state))):
            recommend.append(new_other_state.index[i])
            
    return recommend    

In [10]:
# we have user_table which will give rating to a particular dish
# content based recommeder system
# we have 2 tables with us

#one is food - spices table and another is food-user rating table
#we will use tf-idf approach
#matrix of spice will contain only of 1 and 0's so need to normalize it

#normalization along the row
def spice_norm(spice):
    
    num_attr = np.sum(spice,axis=1)
    doc_vec = []
    for i in range(num_attr.shape[0]):
        if(num_attr[i] != 0): 
            doc_vec.append(np.sqrt(spice[i,:]*1.0/num_attr[i]))
        else:
            doc_vec.append(spice[i,:])
    return np.array(doc_vec)      
    
    
    
def attr_norm(spice):
    
    num_attr = np.sum(spice,axis=0)
    
    doc_vec = []
    for i in range(num_attr.shape[0]):
        if(num_attr[i] != 0):
            doc_vec.append(spice[:,i]*1.0 / num_attr[i])
        else:
            doc_vec.append(spice[:,i])
    return np.array(doc_vec)

def user_v(spice_mat,user):
    user_vec = []
    
    
    #print user[:,0]
    #print spice_mat[:,0]
    
    for i in range(user.shape[1]):
        user_vec.append([np.dot(user[:,i],spice_mat[:,j]) for j in range(spice_mat.shape[1])])
    
    print (user_vec)
    return np.array(user_vec)
    
def weight_predict(spice_mat,user):
        
    user_vec = user_v(spice_mat,user)
    
    preds = []
    #print spice_mat
    for i in range(spice_mat.shape[0]):
        preds.append([np.dot(spice_mat[i,:],user_vec[j,:]) for j in range(user_vec.shape[0])])

    return np.array(preds)
    
    
def idf_weight_pred(user,spice):
    spice_mat = spice_norm(spice)
    attr_wght = attr_norm(spice)
    #print attr_wght
    weighted_spice = []
    
    for i in range(spice_mat.shape[0]):
        weighted_spice.append([np.dot(spice_mat[i,:],attr_wght[:,j]) for j in range(attr_wght.shape[0])])
        
    preds  = weight_predict(np.array(weighted_spice),user)     
    
    return np.array(preds)
    




#print preds.shape

In [11]:
def content_based_reco(user_rating,spice):
    preds = idf_weight_pred(np.array(user_rating),spice)
    outputdf = pd.DataFrame(data = preds,index = user_rating.index ,columns = user_rating.columns)
    return outputdf  

def recommendation_content(user_id,spice_feat,user_rating,non_recom_list,user_feat,outputdf):
    #spice_feat = np.array(spice_feat.iloc[:,0:4])
    #outputdf = content_based_reco(user_rating,spice_feat)
    
    spice = filter_food_cal(spice_feat,non_recom_list,user_feat,user_id)
    #print s_index
    spice.index = spice['food_id']
    output = outputdf[outputdf.index.isin(spice.index)]
    #print user_df.columns
    newoutput = output[user_id].sort_values(ascending = False)
    #print newoutput
    food_to_recommend = list(newoutput[0:3].index)

    return  food_to_recommend




In [12]:

def convert_list(row):
    x = str(row['last_5_days_recommend'])
    y = str(row['last_5_days_bought'])
    if isinstance(x,basestring):
        x = ast.literal_eval(x)
    if isinstance(y,basestring):
        y = ast.literal_eval(y)    
        
    l1 =  list(itertools.chain.from_iterable(x))
    l2 =  list(itertools.chain.from_iterable(y)) 
    #print l1+l2
    return l1+l2 

In [13]:
#for next day
#update the user_feat table for not recommenddation
def change_last_recommend(x,y):
    #print (row['last 5 days recommend'])
    new_recommend = []
    #print type(x) ,y,count 
    if isinstance(x,basestring):
        x = ast.literal_eval(x)
    if isinstance(y,basestring):
        y = ast.literal_eval(y)    
    for i in range(4):
        new_recommend.append(x[i+1])
    new_recommend.append(y)
    #print new_recommend
    return str(new_recommend)

In [22]:
global count
count=0

In [45]:
def recommend_items(user_id,non_recom_list,user,user_feat,spice_feat,user_rating,output_df):
    #print type(non_recom_list)
    l1 = recommendation(user,user_id,non_recom_list,user_feat,spice_feat)
    print 'something1'
    l2 = recommendation_content(user_id,spice_feat,user_rating,non_recom_list,user_feat,output_df)
    
    return str(l1+l2)

In [32]:
def change_type(x):
    if isinstance(x,basestring):
        x = ast.literal_eval(x)
    return x    

In [51]:
def next_day_recommendation(user_feat,new_user_feat,spice_feat,user_rating):
    new_user_feat['last_5_days_recommend'] = new_user_feat.apply(lambda x: change_last_recommend(x['last_5_days_recommend'], x['recommends']), axis=1)
    new_user_feat['last_5_days_bought'] = new_user_feat.apply(lambda x: change_last_recommend(x['last_5_days_bought'], x['bought']), axis=1)
    new_user_feat['not_recommended'] = new_user_feat.apply(convert_list,axis=1)
    print new_user_feat['last_5_days_recommend'][0]
    print new_user_feat['last_5_days_bought'][0] 
    print new_user_feat['not_recommended'][0]
    user_group ,user_feat = clustering_based(user_feat)
    spice = np.array(spice_feat.iloc[: ,0:4])
    outputdf  = content_based_reco(user_rating, spice)
    new_user_feat['not_recommended'] = new_user_feat.apply(lambda x:change_type(x['not_recommended']),axis=1)
    #new_user_feat['not_recommended'].to_csv('not_recommended.csv',encoding='utf-8',index = False)
    #print new_user_feat
    new_user_feat['recommends'] = new_user_feat.apply(lambda x:recommend_items(x['user_guid'],x['not_recommended'],user_group,user_feat,spice_feat,user_rating,outputdf),axis=1)
    print new_user_feat
    return new_user_feat
    
def main_recommendation(user_feat,spice_feat,user_rating):
    new_user_feat = pd.read_csv('new_user_feat.csv')
    next_day = input('type 0 for recommending items or 1 for next_day recommendation') 
    if next_day == 1:
        new_user_feat = next_day_recommendation(user_feat,new_user_feat,spice_feat,user_rating)
        new_user_feat.to_csv('new_user_feat.csv',encoding='utf-8',index = False)       
    else:
        user_id = input('enter the user_id')
        index = np.where(new_user_feat['user_guid']==user_id)[0]
        print new_user_feat.loc[index[0],'recommends']

In [52]:
main_recommendation(user_feat,spice_feat,user_rating)

type 0 for recommending items or 1 for next_day recommendation559
enter the user_id559
[60074, 60089, 60051, 60077, 60051, 60087]
